In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import datetime
import pandas_datareader.data as web
import time

import xgboost as xgb
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

import yfinance as yf

In [ ]:
# Load financial data from Yahoo Finance
start = datetime.datetime(1970, 1, 1)
end = datetime.datetime(2023, 8, 1)
df = yf.download('SPY', start=start, end=end)
df

In [ ]:
# Compute moving average of close
for k in range (1, 20):
    df['MA_' + str(k)] = df['Close'].rolling(k).mean()

df['year'] = df.index.year
df['Target_Close'] = df['Close'].shift(-1)
df.dropna(inplace=True)

train_size = int(len(df) * 0.8)
train, test = df[:train_size], df[train_size:]

features = [col for col in df.columns if 'MA_' in col] + ['Volume'] + ['year']
X_train, y_train = train[features], train['Target_Close']
X_test, y_test = test[features], test['Target_Close']

In [ ]:
lr = LinearRegression()
lr.fit(X_train, y_train)
lr_preds = lr.predict(X_test)

model = xgb.XGBRegressor(objective='reg:squarederror') #Change it
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

mse_xgboost = mean_squared_error(y_test, y_pred)
mse_lr = mean_squared_error(y_test, lr_preds)

print(f"RMSE XGBOOST: {np.sqrt(mse_xgboost)}, RMSE LR: {np.sqrt(mse_lr)}")

In [ ]:
initial_cash = 100_000  # Starting with $100,000 for example
cash = initial_cash
stock_quantity = 0

history = []  # To keep track of our portfolio value over time

# Starting the simulation:
for i in range(len(X_test)):
    # Current day's actual close price
    actual_today = y_test.iloc[i]
    # Tomorrow's predicted price
    predicted_tomorrow = lr_preds[i]
    
    if predicted_tomorrow > actual_today:  # Predicting price will go up
        if cash > 0:  # If we have cash, buy as much stock as we can
            stock_quantity += cash // actual_today
            cash -= stock_quantity * actual_today  # Update our cash after buying
            
    elif predicted_tomorrow < actual_today:  # Predicting price will go down
        if stock_quantity > 0:  # If we have stock, sell all
            cash += stock_quantity * actual_today
            stock_quantity = 0  # Update stock quantity after selling
            
    # Calculate total value (cash + value of stocks we have)
    total_value = cash + (stock_quantity * actual_today)
    history.append(total_value)

# Let's see our final value and profit or loss
final_value = cash + stock_quantity * (y_test.iloc[-1] if stock_quantity > 0 else 0)
profit_or_loss = final_value - initial_cash

print(f"Final Portfolio Value: ${final_value:.2f}")
print(f"Net Profit/Loss: ${profit_or_loss:.2f}")

# Passive strategy

In [ ]:
start = datetime.datetime(2003, 1, 1)
end = datetime.datetime(2023, 8, 10)
df = yf.download('SPY', start=start, end=end)

initial_cash = 3_000
monthly_contribution = 200
total_months = (end.year - start.year) * 12
shares_owned = 0

monthly_prices = df.resample('M').first()

# Lists to store data for plotting
invested_cash = []
portfolio_value = []

for idx, (_, row) in enumerate(monthly_prices.iterrows()):

    if (idx+1)%12 == 0:
        monthly_contribution += 100
        # print(monthly_contribution, start.year + idx//12)
    # Update cash
    cash_available = initial_cash + monthly_contribution
    
    # Buy as many shares as possible
    shares_to_buy = cash_available // row['Open']
    shares_owned += shares_to_buy
    
    # Update cash after buyingA
    initial_cash = cash_available - (shares_to_buy * row['Open'])
    
    # Store data for plotting
    invested_cash.append((invested_cash[-1] + monthly_contribution) if idx>0 else initial_cash + monthly_contribution)
    portfolio_value.append(shares_owned * row['Close'])

# Total invested cash
total_invested = invested_cash[-1]


fig = go.Figure()
fig.add_trace(go.Scatter(x=monthly_prices.index, y=invested_cash,
                         mode='lines', name="Total Cash Invested",
                         line=dict(color='royalblue', width=2, dash='dash')))
fig.add_trace(go.Scatter(x=monthly_prices.index, y=portfolio_value,
                         mode='lines', name="Portfolio Value",
                         line=dict(color='green', width=2)))

fig.update_layout(
    title=f"Dollar-Cost Averaging Strategy: {start.year}-{end.year}",
    xaxis_title="Year",
    yaxis_title="Value ($)",
    font=dict(
        family="Courier New, monospace",
        size=14,
        color="#7f7f7f"
    ))
fig.show()

print(f"Total value by 2023: ${portfolio_value[-1]:,.0f}, last monthly contribution: ${monthly_contribution}")
print(f"Total cash invested by 2023: ${total_invested:,.0f}")